In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import expm_multiply, expm
from scipy.sparse import diags

from os.path import join, dirname
import sys
sys.path.append(join(".", ".."))
from ionq_circuit_utils import *

import sys
sys.path.append(join(".", "..", ".."))
from utils import *

import json
import hashlib

import networkx as nx
from random import shuffle, seed

from braket.devices import LocalSimulator

from qiskit import QuantumCircuit, transpile

# Helper functions

Spatial search circuit

In [ ]:
def get_spatial_search_circuit(N, lamb, gamma, t, r, encoding, use_second_order_pf=False):
    n = num_qubits_per_dim(N, encoding)
    dimension = 2
    instructions = []

    dt = t / r
    np.random.seed(int(t * r))

    if encoding == "unary" or encoding == "antiferromagnetic":
        instructions += get_hadamard_layer(n, dimension)

        for _ in range(r):
            if use_second_order_pf:
                # X rotations (between hadamard)
                for i in range(dimension * n):
                    instructions.append(get_rz(- gamma * dt, i))

                # penalty term
                for i in range(dimension):
                    instructions.append(get_rx(2 * lamb * dt, i * n))
                    if encoding == "unary":
                        instructions.append(get_rx(-2 * lamb * dt, int((i + 1) * n - 1)))
                    if encoding == "antiferromagnetic":
                        instructions.append(get_rx((-1) ** (n) * 2 * lamb * dt, int((i + 1) * n - 1)))
                    
                    for j in np.arange(i * n, (i + 1) * n - 1):
                        if encoding == "unary":
                            instructions.append(get_rxx(-2 * lamb * dt, [int(j), int(j+1)]))
                        elif encoding == "antiferromagnetic":
                            instructions.append(get_rxx(2 * lamb * dt, [int(j), int(j+1)]))

                # oracle term
                instructions.append(get_rxx(dt / 2, [int(n-1), int(n)]))
                instructions.append(get_rx(dt / 2, n-1))
                instructions.append(get_rx(- dt / 2, n))

                # laplacian correction term
                for i in range(dimension):
                    instructions.append(get_rx(-gamma * dt, i * n))
                    if encoding == "unary":
                        instructions.append(get_rx(gamma * dt, int((i + 1) * n - 1)))
                    elif encoding == "antiferromagnetic":
                        instructions.append(get_rx((-1) ** (n+1) * gamma * dt, int((i + 1) * n - 1)))
                
                # X rotations (between hadamard)
                for i in range(dimension * n):
                    instructions.append(get_rz(- gamma * dt, i))
            else:
                # Use randomized first order
                if np.random.rand() < 0.5:
                    # X rotations (between hadamard)
                    for i in range(dimension * n):
                        instructions.append(get_rz(- 2 * gamma * dt, i))

                    # penalty term
                    for i in range(dimension):
                        instructions.append(get_rx(2 * lamb * dt, i * n))
                        if encoding == "unary":
                            instructions.append(get_rx(-2 * lamb * dt, int((i + 1) * n - 1)))
                        if encoding == "antiferromagnetic":
                            instructions.append(get_rx((-1) ** (n) * 2 * lamb * dt, int((i + 1) * n - 1)))
                        
                        for j in np.arange(i * n, (i + 1) * n - 1):
                            if encoding == "unary":
                                instructions.append(get_rxx(-2 * lamb * dt, [int(j), int(j+1)]))
                            elif encoding == "antiferromagnetic":
                                instructions.append(get_rxx(2 * lamb * dt, [int(j), int(j+1)]))

                    # oracle term
                    instructions.append(get_rxx(dt / 2, [int(n-1), int(n)]))
                    instructions.append(get_rx(dt / 2, n-1))
                    instructions.append(get_rx(- dt / 2, n))

                    # laplacian correction term
                    for i in range(dimension):
                        instructions.append(get_rx(-gamma * dt, i * n))
                        if encoding == "unary":
                            instructions.append(get_rx(gamma * dt, int((i + 1) * n - 1)))
                        elif encoding == "antiferromagnetic":
                            instructions.append(get_rx((-1) ** (n+1) * gamma * dt, int((i + 1) * n - 1)))
                else:
                    # laplacian correction term
                    for i in range(dimension):
                        instructions.append(get_rx(-gamma * dt, i * n))
                        if encoding == "unary":
                            instructions.append(get_rx(gamma * dt, int((i + 1) * n - 1)))
                        elif encoding == "antiferromagnetic":
                            instructions.append(get_rx((-1) ** (n+1) * gamma * dt, int((i + 1) * n - 1)))

                    # oracle term
                    instructions.append(get_rxx(dt / 2, [int(n-1), int(n)]))
                    instructions.append(get_rx(dt / 2, n-1))
                    instructions.append(get_rx(- dt / 2, n))

                    # penalty term
                    for i in range(dimension):
                        instructions.append(get_rx(2 * lamb * dt, i * n))
                        if encoding == "unary":
                            instructions.append(get_rx(-2 * lamb * dt, int((i + 1) * n - 1)))
                        if encoding == "antiferromagnetic":
                            instructions.append(get_rx((-1) ** (n) * 2 * lamb * dt, int((i + 1) * n - 1)))
                        
                        for j in np.arange(i * n, (i + 1) * n - 1)[::-1]:
                            if encoding == "unary":
                                instructions.append(get_rxx(-2 * lamb * dt, [int(j), int(j+1)]))
                            elif encoding == "antiferromagnetic":
                                instructions.append(get_rxx(2 * lamb * dt, [int(j), int(j+1)]))
                                
                    # X rotations (between hadamard)
                    for i in range(dimension * n):
                        instructions.append(get_rz(- 2 * gamma * dt, i))
        
        instructions += get_hadamard_layer(n, dimension)

        return instructions
    elif encoding == "one-hot":
        marked_vertex_index_1 = N-1
        marked_vertex_index_2 = 0
        for _ in range(r):
            if use_second_order_pf:
                # Laplacian term
                for i in range(dimension):
                    for j in np.arange(i * n, (i + 1) * n - 1, 2):
                        instructions.append(get_rxx(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                        instructions.append(get_ryy(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                    for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                        instructions.append(get_rxx(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                        instructions.append(get_ryy(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                        
                # Laplacian correction term
                for i in range(dimension):
                    instructions.append(get_rz(gamma * dt, int(i * n)))
                    instructions.append(get_rz(gamma * dt, int((i + 1) * n - 1)))

                # Oracle term
                instructions.append(get_rzz(- dt / 2, [marked_vertex_index_1, n + marked_vertex_index_2]))
                instructions.append(get_rz(dt / 2, marked_vertex_index_1))
                instructions.append(get_rz(dt / 2, n + marked_vertex_index_2))

                # Laplacian term
                for i in range(dimension):
                    for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                        instructions.append(get_rxx(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                        instructions.append(get_ryy(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                    for j in np.arange(i * n, (i + 1) * n - 1, 2):
                        instructions.append(get_rxx(- 0.5 * gamma * dt, [int(j), int(j+1)]))
                        instructions.append(get_ryy(- 0.5 * gamma * dt, [int(j), int(j+1)]))
            else:
                if np.random.rand() < 0.5:
                    # Laplacian term
                    for i in range(dimension):
                        for j in np.arange(i * n, (i + 1) * n - 1, 2):
                            instructions.append(get_rxx(- gamma * dt, [int(j), int(j+1)]))
                            instructions.append(get_ryy(- gamma * dt, [int(j), int(j+1)]))
                        for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                            instructions.append(get_rxx(- gamma * dt, [int(j), int(j+1)]))
                            instructions.append(get_ryy(- gamma * dt, [int(j), int(j+1)]))
                            
                    # Laplacian correction term
                    for i in range(dimension):
                        instructions.append(get_rz(gamma * dt, int(i * n)))
                        instructions.append(get_rz(gamma * dt, int((i + 1) * n - 1)))

                    # Oracle term
                    instructions.append(get_rzz(- dt / 2, [marked_vertex_index_1, n + marked_vertex_index_2]))
                    instructions.append(get_rz(dt / 2, marked_vertex_index_1))
                    instructions.append(get_rz(dt / 2, n + marked_vertex_index_2))

                else:
                    # Oracle term
                    instructions.append(get_rzz(- dt / 2, [marked_vertex_index_1, n + marked_vertex_index_2]))
                    instructions.append(get_rz(dt / 2, marked_vertex_index_1))
                    instructions.append(get_rz(dt / 2, n + marked_vertex_index_2))

                    # Laplacian correction term
                    for i in range(dimension):
                        instructions.append(get_rz(gamma * dt, int(i * n)))
                        instructions.append(get_rz(gamma * dt, int((i + 1) * n - 1)))

                    # Laplacian term
                    for i in range(dimension):
                        for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                            instructions.append(get_rxx(- gamma * dt, [int(j), int(j+1)]))
                            instructions.append(get_ryy(- gamma * dt, [int(j), int(j+1)]))
                        for j in np.arange(i * n, (i + 1) * n - 1, 2):
                            instructions.append(get_rxx(- gamma * dt, [int(j), int(j+1)]))
                            instructions.append(get_ryy(- gamma * dt, [int(j), int(j+1)]))
        
        return instructions
    else:
        raise ValueError("Encoding not supported")

In [ ]:
def run_ss(N, dimension, encoding, bitstrings, num_time_points, t_vals, task_name, 
           gamma, r, num_shots, device, save_dir, use_real_machine, 
           qpu_job_ids_filename, lamb=None, use_second_order_pf=False,
           optimize_circuit=False):
    n = num_qubits_per_dim(N, encoding)

    job_ids = []

    # unnormalized probabilities
    sim_freq = np.zeros((num_time_points, N ** dimension))

    amplitudes_input = np.ones(N)
    amplitudes_input /= np.linalg.norm(amplitudes_input)

    for i, t in enumerate(t_vals):
        
        print(f"Unitless time: {t : 0.3f}")

        # use braket simulator and get amplitudes of state vector
        instructions = state_prep_circuit(N, dimension, amplitudes_input, encoding)
        instructions += get_spatial_search_circuit(N, lamb, gamma, t, r, encoding, use_second_order_pf)
        if optimize_circuit:
            # Optimize with Qiskit
            compiled_circuit = transpile(get_qiskit_circuit(n * dimension, instructions), basis_gates=['rxx', 'rx', 'ry', 'rz'], optimization_level=3)
            instructions = get_circuit_from_qiskit(compiled_circuit)
            
        if use_real_machine:
            
            # Create the job json and save it
            job = get_ionq_job_json(task_name, N, dimension, num_shots, device, encoding, instructions, use_native_gates=True)
            
            print(f"Saving in {save_dir}")
            with open(join(save_dir, f"job_{i}.json"), "w") as f:
                json.dump(job, f, default=int)

            # Send the job and get the job id
            job_id = send_job(job)
            print("Job id:", job_id)
            job_ids.append(job_id)
        else:
            
            native_instructions, qubit_phase = get_native_circuit(dimension * n, instructions)
            one_qubit_gate_count, two_qubit_gate_count = get_native_gate_counts(native_instructions)
            print(f"1q gates: {one_qubit_gate_count}, 2q gates: {two_qubit_gate_count}")
            circuit = get_braket_native_circuit(native_instructions)
            for j in range(n * dimension):
                circuit.rz(j, -qubit_phase[j] * (2 * np.pi))

            circuit.amplitude(state=bitstrings)

            # Run on simulator
            task = device.run(circuit)
            # occurrences = get_occurrences(N, task.result().measurements, num_shots, dimension, encoding, periodic=False)
            # sim_freq[i] = occurrences.flatten() / np.sum(occurrences)

            amplitudes_res = task.result().values[0]
            for j in range(N ** dimension):
                sim_freq[i,j] = np.abs(amplitudes_res[bitstrings[j]]) ** 2

    if use_real_machine:
        print("Saving IonQ job ids")
        with open(join(save_dir, qpu_job_ids_filename), "w") as f:
            json.dump(job_ids, f)
            f.close()
    else:
        return sim_freq

# Spatial search

In [ ]:
DATA_DIR = "experiment_data"
TASK_DIR = "spatial_search"

CURR_DIR = join("..", "..", "..", DATA_DIR)
check_and_make_dir(CURR_DIR)
CURR_DIR = join(CURR_DIR, TASK_DIR)
check_and_make_dir(CURR_DIR)

print(CURR_DIR)

use_real_machine = False
if use_real_machine:
    device = "qpu.aria-1"
    print("Device:", device)
else:
    device = LocalSimulator()
    print(f"Using {device.name}")

In [ ]:
N = 4
encoding = "unary"
r = 12
lamb = 2

# N = 5
# encoding = "one-hot"
# r = 5
# lamb = None

dimension = 2
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords(N, dimension, encoding, periodic=False)
bitstrings = get_bitstrings(N, dimension, encoding)

num_time_points = 13
num_shots = 200

def get_T_2d(N, gamma, H_spatial_search):
    dimension = 2
    T = N ** dimension
    num_time_points = 256
    t_vals = np.linspace(0, T, num_time_points)
    p = 4 * (np.log(N) / N) ** 2
    psi_0 = np.ones(N ** dimension)
    psi_0 /= np.linalg.norm(psi_0)

    psi = expm_multiply(-1j * H_spatial_search, psi_0, start=0, stop=T, num=num_time_points)
    dist = np.abs(psi) ** 2

    return t_vals[np.argmax(dist[:,-N] >= p)]

# Compute the optimal gamma
L = get_laplacian_lattice(N, dimension)
marked_vertex_1 = np.zeros(N)
marked_vertex_1[0] = 1
marked_vertex_2 = np.zeros(N)
marked_vertex_2[N-1] = 1

marked_vertex = np.kron(marked_vertex_2, marked_vertex_1)
H_oracle = -csc_matrix(np.outer(marked_vertex, marked_vertex))
# Sign is flipped here; this function minimizes the difference between the two largest eigenvalues of gamma * L + H_oracle
gamma = scipy_get_optimal_gamma(L, -H_oracle, 0.3)
print(f"gamma = {gamma}")
H_spatial_search = - gamma * L + H_oracle
T = get_T_2d(N, gamma, H_spatial_search)
print(f"T = {T : 0.2f}")
t_vals = np.linspace(0, T, num_time_points)

if encoding == "unary":
    optimize_circuit = True
else:
    optimize_circuit = False
    
use_error_mitigation = False
# Second order PF generally seems to give better results than randomized first order Trotter
use_second_order_pf = True

if use_error_mitigation:
    assert num_shots >= 500, "Number of shots should be at least 500"
if not encoding == "one-hot":
    assert lamb is not None
    
if encoding == "one-hot":
    experiment_info = {
        "N": N,
        "dimension": dimension,
        "encoding": encoding,
        "T": T,
        "num_time_points": num_time_points,
        "r": r,
        "num_shots": num_shots,
        "optimize_circuit": optimize_circuit,
        "use_error_mitigation": use_error_mitigation,
        "use_second_order_pf": use_second_order_pf
    }
else:
    experiment_info = {
        "N": N,
        "dimension": dimension,
        "encoding": encoding,
        "T": T,
        "num_time_points": num_time_points,
        "lamb": lamb,
        "r": r,
        "num_shots": num_shots,
        "optimize_circuit": optimize_circuit,
        "use_error_mitigation": use_error_mitigation,
        "use_second_order_pf": use_second_order_pf
    }

hash_str = hashlib.md5(json.dumps(experiment_info).encode("utf-8")).hexdigest()
SAVE_DIR = join(CURR_DIR, hash_str)
check_and_make_dir(SAVE_DIR)

print("Save dir:", SAVE_DIR)

with open(join(SAVE_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

name = "spatial_search"
qpu_job_ids_filename = f'job_ids_qpu.json'

In [ ]:
amplitudes_input = np.ones(N)
amplitudes_input /= np.linalg.norm(amplitudes_input)
instructions = state_prep_circuit(N, dimension, amplitudes_input, encoding)
instructions += get_spatial_search_circuit(N, lamb, gamma, T, r, encoding, use_second_order_pf)

if optimize_circuit:
    # Optimize with Qiskit
    compiled_circuit = transpile(get_qiskit_circuit(n * dimension, instructions), basis_gates=['rxx', 'rx', 'ry', 'rz'], optimization_level=3)
    instructions = get_circuit_from_qiskit(compiled_circuit)

job = get_ionq_job_json("", N, dimension, num_shots, device, encoding, instructions, use_native_gates=True)

Submit tasks

In [ ]:
if use_real_machine:
    run_ss(N, dimension, encoding, bitstrings, num_time_points, t_vals, TASK_DIR, 
           gamma, r, num_shots, device, SAVE_DIR, use_real_machine, 
           qpu_job_ids_filename, lamb=lamb, use_second_order_pf=use_second_order_pf,
           optimize_circuit=optimize_circuit)
else:
    sim_freq = run_ss(N, dimension, encoding, bitstrings, num_time_points, t_vals, TASK_DIR, 
           gamma, r, num_shots, device, SAVE_DIR, use_real_machine, 
           qpu_job_ids_filename, lamb=lamb, use_second_order_pf=use_second_order_pf,
           optimize_circuit=optimize_circuit)

Get data from completed tasks

In [ ]:
ionq_freq = get_results(join(SAVE_DIR, qpu_job_ids_filename), num_time_points, codewords, use_error_mitigation=False)

Post processing and figures

In [ ]:
# Ideal
psi_0 = np.ones(N ** dimension, dtype=np.complex64)
psi_0 /= np.linalg.norm(psi_0)

H = -gamma * L + H_oracle
num_time_points_ideal = num_time_points
psi = expm_multiply(-1j * H, psi_0, start=0, stop=T, num=num_time_points_ideal)
ideal_dist = np.abs(psi) ** 2
ideal_success_probability = ideal_dist[:, N ** dimension - N]

In [ ]:
sim_freq_normalized = np.zeros_like(sim_freq)
for i in range(num_time_points):
    if np.sum(sim_freq[i]) > 0:
        sim_freq_normalized[i] = sim_freq[i] / np.sum(sim_freq[i])

ionq_freq_normalized = np.zeros_like(ionq_freq)
for i in range(num_time_points):
    if np.sum(ionq_freq[i]) > 0:
        ionq_freq_normalized[i] = ionq_freq[i] / np.sum(ionq_freq[i])

num_samples_subspace_ionq = np.sum(ionq_freq, axis=1) * num_shots

valid_points_sim = np.sum(sim_freq, axis=1) > 0
valid_points_ionq = num_samples_subspace_ionq > 0

success_prob_sim = sim_freq_normalized[:,-N]
success_prob_ionq = ionq_freq_normalized[:,-N]
err_ionq = np.sqrt(success_prob_ionq * (1 - success_prob_ionq) / (num_samples_subspace_ionq - 1))


In [ ]:
plt.matshow(ionq_freq_normalized[-1,:].reshape(N,N))
plt.title(f"{N} by {N} lattice ({encoding})")
plt.colorbar()
plt.show()

In [ ]:
check_and_make_dir(f"{N}_by_{N}")

with open(join(f"{N}_by_{N}", "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

np.savez(join(f"{N}_by_{N}", "data.npz"), 
        ideal_dist=ideal_dist,
        sim_freq=sim_freq,
        ionq_freq=ionq_freq,
        ionq_freq_normalized=ionq_freq_normalized,
        num_samples_subspace_ionq=num_samples_subspace_ionq)